In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 도구

여기서는 고급 도구 사용법을 살펴보겠습니다.

> [!TIP]
> 에이전트 구축이 처음이라면 먼저 [에이전트 소개](https://huggingface.co/docs/smolagents/main/ko/tutorials/../conceptual_guides/intro_agents)와 [smolagents 둘러보기](https://huggingface.co/docs/smolagents/main/ko/tutorials/../guided_tour)를 읽어보세요.

### 도구란 무엇이며 어떻게 만드나요?

도구는 대부분 LLM이 에이전트 시스템에서 사용할 수 있는 함수입니다.

하지만 이를 사용하려면 LLM에 API(이름, 도구 설명, 입력 유형 및 설명, 출력 유형)가 제공되어야 합니다.

따라서 함수만으로는 충분하지 않습니다. 클래스여야 합니다.

따라서 핵심적으로 도구는 LLM이 사용 방법을 이해하는 데 도움이 되는 메타데이터로 함수를 래핑하는 클래스입니다.

다음과 같이 보입니다.

In [ ]:
from smolagents import Tool

class HFModelDownloadsTool(Tool):
    name = "model_download_counter"
    description = """
    이 도구는 Hugging Face Hub에서 지정된 작업의 가장 많이 다운로드된 모델을 반환합니다.
    체크포인트 이름을 반환합니다."""
    inputs = {
        "task": {
            "type": "string",
            "description": "작업 범주 (예: 텍스트 분류, 깊이 추정 등)",
        }
    }
    output_type = "string"

    def forward(self, task: str):
        from huggingface_hub import list_models

        model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
        return model.id

model_downloads_tool = HFModelDownloadsTool()

사용자 지정 도구는 유용한 메서드를 상속하기 위해 [Tool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool)을 하위 클래스로 만듭니다. 하위 클래스는 다음도 정의합니다.
- 도구 자체의 이름에 해당하는 `name` 속성입니다. 이름은 일반적으로 도구가 수행하는 작업을 설명합니다. 코드가 작업에 대해 가장 많이 다운로드된 모델을 반환하므로 `model_download_counter`라고 이름을 지정하겠습니다.
- `description` 속성은 에이전트의 시스템 프롬프트를 채우는 데 사용됩니다.
- `"type"` 및 `"description"` 키가 있는 사전인 `inputs` 속성입니다. Python 인터프리터가 입력에 대해 정보에 입각한 선택을 하는 데 도움이 되는 정보가 포함되어 있습니다.
- `output_type` 속성은 출력 유형을 지정합니다. `inputs` 및 `output_type` 모두에 대한 유형은 [Pydantic 형식](https://docs.pydantic.dev/latest/concepts/json_schema/#generating-json-schema)이어야 하며 다음 중 하나일 수 있습니다. `~AUTHORIZED_TYPES()`.
- 실행할 추론 코드가 포함된 `forward` 메서드입니다.

그리고 이것이 에이전트에서 사용되는 데 필요한 전부입니다!

도구를 만드는 또 다른 방법이 있습니다. [guided_tour](https://huggingface.co/docs/smolagents/main/ko/tutorials/../guided_tour)에서는 `@tool` 데코레이터를 사용하여 도구를 구현했습니다. [tool()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.tool) 데코레이터는 간단한 도구를 정의하는 권장 방법이지만 때로는 이보다 더 많은 것이 필요합니다. 예를 들어 더 명확성을 위해 클래스에서 여러 메서드를 사용하거나 추가 클래스 속성을 사용하는 경우입니다.

이 경우 위에서 설명한 대로 [Tool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool)을 하위 클래스로 만들어 도구를 빌드할 수 있습니다.

### 도구를 허브에 공유하기

도구에서 [push_to_hub()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool.push_to_hub)를 호출하여 사용자 지정 도구를 허브에 Space 리포지토리로 공유할 수 있습니다. 허브에 해당 리포지토리를 만들고 읽기 액세스 권한이 있는 토큰을 사용하고 있는지 확인하십시오.

In [ ]:
model_downloads_tool.push_to_hub("{your_username}/hf-model-downloads", token="<YOUR_HUGGINGFACEHUB_API_TOKEN>")

허브에 푸시하려면 도구가 몇 가지 규칙을 따라야 합니다.
- 모든 메서드는 자체 포함되어야 합니다. 즉, 인수에서 가져온 변수를 사용합니다.
- 위 사항에 따라 **모든 가져오기는 도구 함수 내에서 직접 정의해야 합니다.** 그렇지 않으면 사용자 지정 도구로 [save()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool.save) 또는 [push_to_hub()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool.push_to_hub)를 호출하려고 할 때 오류가 발생합니다.
- `__init__` 메서드를 하위 클래스로 만드는 경우 `self` 이외의 다른 인수를 제공할 수 없습니다. 이는 특정 도구 인스턴스 초기화 중에 설정된 인수를 추적하기 어렵기 때문이며, 이로 인해 허브에 제대로 공유할 수 없습니다. 어쨌든 특정 클래스를 만드는 아이디어는 하드 코딩해야 하는 모든 항목에 대해 이미 클래스 속성을 설정할 수 있다는 것입니다(`class YourTool(Tool):` 줄 바로 아래에 `your_variable=(...)` 설정). 물론 `self.your_variable`에 항목을 할당하여 코드 어디에서나 클래스 속성을 만들 수 있습니다.


도구가 허브에 푸시되면 시각화할 수 있습니다. [여기](https://huggingface.co/spaces/m-ric/hf-model-downloads)에 제가 푸시한 `model_downloads_tool`이 있습니다. 멋진 gradio 인터페이스가 있습니다.

도구 파일을 자세히 살펴보면 모든 도구 논리가 [tool.py](https://huggingface.co/spaces/m-ric/hf-model-downloads/blob/main/tool.py) 아래에 있음을 알 수 있습니다. 다른 사람이 공유한 도구를 검사할 수 있는 곳입니다.

그런 다음 [load_tool()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.load_tool)로 도구를 로드하거나 [from_hub()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool.from_hub)로 만들고 에이전트의 `tools` 매개변수에 전달할 수 있습니다.
도구를 실행하는 것은 사용자 지정 코드를 실행하는 것을 의미하므로 리포지토리를 신뢰하는지 확인해야 합니다. 따라서 허브에서 도구를 로드하려면 `trust_remote_code=True`를 전달해야 합니다.

In [ ]:
from smolagents import load_tool, CodeAgent

model_download_tool = load_tool(
    "{your_username}/hf-model-downloads",
    trust_remote_code=True
)

### MCP 서버에서 도구 사용

`MCPClient`를 사용하면 MCP 서버에서 도구를 로드하고 연결 및 도구 관리를 완벽하게 제어할 수 있습니다.

stdio 기반 MCP 서버의 경우:

In [ ]:
from smolagents import MCPClient, CodeAgent
from mcp import StdioServerParameters
import os

server_parameters = StdioServerParameters(
    command="uvx",  # uvx를 사용하면 종속성을 사용할 수 있도록 보장합니다.
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with MCPClient(server_parameters) as tools:
    agent = CodeAgent(tools=tools, model=model, add_base_tools=True)
    agent.run("COVID-19 치료에 대한 최신 연구를 찾아주세요.")

스트리밍 가능한 HTTP 기반 MCP 서버의 경우:

In [ ]:
from smolagents import MCPClient, CodeAgent

with MCPClient({"url": "http://127.0.0.1:8000/mcp", "transport": "streamable-http"}) as tools:
    agent = CodeAgent(tools=tools, model=model, add_base_tools=True)
    agent.run("숙취 해소 방법을 찾아주세요.")

try...finally 패턴을 사용하여 연결 수명 주기를 수동으로 관리할 수도 있습니다.

In [ ]:
from smolagents import MCPClient, CodeAgent
from mcp import StdioServerParameters
import os

# 서버 매개변수 초기화
server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

# 연결 수동 관리
try:
    mcp_client = MCPClient(server_parameters)
    tools = mcp_client.get_tools()

    # 에이전트와 함께 도구 사용
    agent = CodeAgent(tools=tools, model=model, add_base_tools=True)
    result = agent.run("알츠하이머병에 대한 최근 치료 접근법은 무엇인가요?")

    # 필요에 따라 결과 처리
    print(f"에이전트 응답: {result}")
finally:
    # 항상 연결이 제대로 닫혔는지 확인
    mcp_client.disconnect()

서버 매개변수 목록을 전달하여 한 번에 여러 MCP 서버에 연결할 수도 있습니다.

In [ ]:
from smolagents import MCPClient, CodeAgent
from mcp import StdioServerParameters
import os

server_params1 = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

server_params2 = {"url": "http://127.0.0.1:8000/sse"}

with MCPClient([server_params1, server_params2]) as tools:
    agent = CodeAgent(tools=tools, model=model, add_base_tools=True)
    agent.run("최신 연구를 분석하고 두통에 대한 치료법을 제안해주세요.")

> [!WARNING]
> **보안 경고:** MCP 서버를 사용하면 보안 위험이 따릅니다.
> - **신뢰는 필수적입니다:** 특히 프로덕션 환경의 경우 연결하기 전에 항상 MCP 서버의 출처와 무결성을 확인하십시오. 악성 서버는 컴퓨터에서 유해한 코드를 실행할 수 있습니다.
> - **stdio 기반 MCP 서버**는 항상 컴퓨터에서 코드를 실행합니다(이것이 의도된 기능입니다).
> - **스트리밍 가능한 HTTP 기반 MCP 서버**의 경우 원격 MCP 서버는 컴퓨터에서 코드를 실행할 수 없지만 여전히 주의해서 진행하십시오.

### 스페이스를 도구로 가져오기

[Tool.from_space()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.Tool.from_space) 메서드를 사용하여 허브에서 Gradio 스페이스를 도구로 직접 가져올 수 있습니다!

허브에 있는 스페이스의 ID, 이름 및 에이전트가 도구가 수행하는 작업을 이해하는 데 도움이 되는 설명만 제공하면 됩니다. 내부적으로 이는 [`gradio-client`](https://pypi.org/project/gradio-client/) 라이브러리를 사용하여 스페이스를 호출합니다.

예를 들어 허브에서 [FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev) 스페이스를 가져와 이미지를 생성해 보겠습니다.

In [ ]:
image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="프롬프트에서 이미지 생성"
)

image_generation_tool("화창한 해변")

그리고 짜잔, 여기 이미지가 있습니다! 🏖️

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/sunny_beach.webp">

그런 다음 다른 도구와 마찬가지로 이 도구를 사용할 수 있습니다. 예를 들어 `토끼가 우주복을 입고 있는 모습`이라는 프롬프트를 개선하고 해당 이미지를 생성해 보겠습니다. 이 예제는 에이전트에 추가 인수를 전달하는 방법도 보여줍니다.

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")
agent = CodeAgent(tools=[image_generation_tool], model=model)

agent.run(
    "이 프롬프트를 개선한 다음 해당 이미지를 생성하십시오.", additional_args={'user_prompt': '토끼가 우주복을 입고 있는 모습'}
)

```text
=== 에이전트 생각:
개선된 프롬프트는 "밝은 파란색 우주복을 입은 토끼가 밝은 주황색 일몰 아래 달 표면에 있고 배경에는 지구가 보이는 모습"일 수 있습니다.

이제 프롬프트를 개선했으므로 이미지 생성기 도구를 사용하여 이 프롬프트를 기반으로 이미지를 생성할 수 있습니다.
>>> 에이전트가 아래 코드를 실행 중입니다.
image = image_generator(prompt="밝은 파란색 우주복을 입은 토끼가 밝은 주황색 일몰 아래 달 표면에 있고 배경에는 지구가 보이는 모습")
final_answer(image)
```

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/rabbit_spacesuit_flux.webp">

정말 멋지지 않나요? 🤩

### LangChain 도구 사용

우리는 Langchain을 좋아하며 매우 매력적인 도구 모음을 가지고 있다고 생각합니다.
LangChain에서 도구를 가져오려면 `from_langchain()` 메서드를 사용하십시오.

LangChain 웹 검색 도구를 사용하여 소개의 검색 결과를 다시 만드는 방법은 다음과 같습니다.
이 도구가 제대로 작동하려면 `pip install langchain google-search-results -q`가 필요합니다.

In [ ]:
from langchain.agents import load_tools

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

agent = CodeAgent(tools=[search_tool], model=model)

agent.run("BERT 기본 인코더에는 Attention is All You Need에서 제안한 아키텍처의 인코더보다 얼마나 많은 블록(레이어라고도 함)이 더 있습니까?")

### 에이전트 도구 상자 관리

에이전트의 도구 상자는 표준 사전이므로 `agent.tools` 속성에서 도구를 추가하거나 교체하여 관리할 수 있습니다.

기본 도구 상자만으로 초기화된 기존 에이전트에 `model_download_tool`을 추가해 보겠습니다.

In [ ]:
from smolagents import InferenceClientModel

model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct")

agent = CodeAgent(tools=[], model=model, add_base_tools=True)
agent.tools[model_download_tool.name] = model_download_tool

이제 새 도구를 활용할 수 있습니다.

In [ ]:
agent.run(
    "Hugging Face Hub의 'text-to-video' 작업에서 가장 많이 다운로드된 모델의 이름을 알려주시되 글자를 거꾸로 뒤집어주세요."
)

> [!TIP]
> 에이전트에 너무 많은 도구를 추가하지 않도록 주의하십시오. 약한 LLM 엔진에 과부하가 걸릴 수 있습니다.

### 도구 모음 사용

[ToolCollection](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.ToolCollection)을 사용하여 도구 모음을 활용할 수 있습니다. 허브의 모음 또는 MCP 서버 도구를 로드하는 것을 지원합니다.

#### 모든 MCP 서버의 도구 모음

[glama.ai](https://glama.ai/mcp/servers) 또는 [smithery.ai](https://smithery.ai/)에서 사용 가능한 수백 개의 MCP 서버에서 도구를 활용하십시오.

MCP 서버 도구는 [ToolCollection.from_mcp()](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.ToolCollection.from_mcp)로 로드할 수 있습니다.

> [!WARNING]
> **보안 경고:** `MCPClient`를 직접 사용할 때 언급된 것과 동일한 보안 경고가 적용됩니다.


stdio 기반 MCP 서버의 경우 서버 매개변수를 `mcp.StdioServerParameters`의 인스턴스로 전달하십시오.

In [ ]:
from smolagents import ToolCollection, CodeAgent
from mcp import StdioServerParameters

server_parameters = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], model=model, add_base_tools=True)
    agent.run("숙취 해소 방법을 찾아주세요.")

스트리밍 가능한 HTTP 기반 MCP 서버의 경우 매개변수가 있는 사전을 `mcp.client.streamable_http.streamablehttp_client`에 전달하고 `transport` 키에 `"streamable-http"` 값을 추가하기만 하면 됩니다.

In [ ]:
from smolagents import ToolCollection, CodeAgent

with ToolCollection.from_mcp({"url": "http://127.0.0.1:8000/mcp", "transport": "streamable-http"}, trust_remote_code=True) as tool_collection:
    agent = CodeAgent(tools=[*tool_collection.tools], add_base_tools=True)
    agent.run("숙취 해소 방법을 찾아주세요.")

#### 허브의 컬렉션에서 도구 컬렉션 가져오기

사용하려는 컬렉션의 슬러그로 활용할 수 있습니다.
그런 다음 목록으로 전달하여 에이전트를 초기화하고 사용을 시작하십시오!

In [ ]:
from smolagents import ToolCollection, CodeAgent

image_tool_collection = ToolCollection.from_hub(
    collection_slug="huggingface-tools/diffusion-tools-6630bb19a942c2306a2cdb6f",
    token="<YOUR_HUGGINGFACEHUB_API_TOKEN>"
)
agent = CodeAgent(tools=[*image_tool_collection.tools], model=model, add_base_tools=True)

agent.run("강과 호수 그림을 그려주세요.")

시작 속도를 높이기 위해 에이전트가 호출하는 경우에만 도구가 로드됩니다.